In [1]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


import time
import tqdm
import jpype
import ast

In [29]:
# 전처리, 명사추출, 키워드추출, 불용어후처리, 닉네임추출까지 마친 데이터

import pandas as pd

folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_after_cleaning_nickname.csv')
comments_df = pd.read_csv(file_path)

In [37]:
sample_df = comments_df.copy()

# 특정 그룹에 대해 분석 수행
sample_df = sample_df[sample_df['Group']=='NMIXX']

# csv 파일로 저장되면서 문자열로 변형된 word_list 컬럼을 다시 리스트로 변환
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: " ".join(x))
print(len(sample_df))

# word_list가 빈 문자열인 행 제거
sample_df = sample_df[sample_df['word_list'] != '']  # 빈 문자열인 행 제거
print(len(sample_df))

#  members 컬럼을 다시 리스트로 변환
sample_df['members'] = sample_df['members'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

23511
20160


In [38]:
sample_df

,Group,Title,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,korean,word_list,members
0,NMIXX,별별별 (See that),node,UgyiK-QaarUUumBCjUJ4AaABAg,NaN,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,@lumi1216,2024-10-01T13:40:14Z,4,True,중독 진심 노래 의문 엔믹스,NaN
1,NMIXX,별별별 (See that),node,Ugw_6c4z7iSqIjzfgZB4AaABAg,NaN,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,@써니노,2024-10-01T09:56:20Z,1,True,기도 아쉬움 대쉬 금도 대중성 노래,NaN
2,NMIXX,별별별 (See that),node,UgxLdDuk-Ji3RKARe_h4AaABAg,NaN,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,@고해진-i4g,2024-10-01T05:16:15Z,4,True,본부장 엘범 천만 클릭 연락 연결 취해 피씨 뮤비 노래 뷰수 수정 재생 조치 벌써 ...,NaN
3,NMIXX,별별별 (See that),node,Ugz-Tnp-d-MLAQSXvQF4AaABAg,NaN,이야 노래 종나 멋있네,@박종한-i9u,2024-09-30T18:33:22Z,10,True,노래,NaN
4,NMIXX,별별별 (See that),node,UgwlfRfRmVnAsHIREut4AaABAg,NaN,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,@egi3581,2024-09-30T17:22:58Z,6,True,걸그룹 멤버 빵빵 노력 묵자 얼마나 세대 외모췍 더더 리더 해원,[해원]
...,...,...,...,...,...,...,...,...,...,...,...,...
24907,NMIXX,O.O,reply,NaN,UgylN03F4f5dGCSKOhx4AaABAg,1곡에 3가지 테마느낌,@flame_flower,2022-02-22T09:03:56Z,0,True,테마,NaN
24910,NMIXX,O.O,node,UgxOM0jR6upi_b8C3EN4AaABAg,NaN,누나들너무예뻐요,@김도우-e5v,2022-02-22T09:00:19Z,0,True,누나,NaN
24912,NMIXX,O.O,node,UgxlQLnk8GwvaEpmLHx4AaABAg,NaN,ㅈㄴ짜 사랑해요ㅜㅜ,@_tkdgml5710,2022-02-22T09:00:18Z,0,True,사랑해,NaN
24914,NMIXX,O.O,node,UgxlHh5mi9d93XW0jpt4AaABAg,NaN,트와이스 데뷔할때도 늦었는데 내가 엔믹스 데뷔를 이렇게까지 챙기게 될줄이야...,@shiri4521,2022-02-22T09:00:17Z,1,True,트와이스 데뷔 엔믹스,NaN


In [39]:
# TF-IDF 벡터화 (전체 데이터에 대해 벡터화 수행)
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # 최대 1000개의 단어만 사용
tfidf_matrix = tfidf_vectorizer.fit_transform(sample_df['word_list'])

# LDA 모델 학습 (전체 데이터에 대해 학습)
n_topics = 10  # 원하는 토픽 수 설정
lda = LatentDirichletAllocation(n_components=n_topics, 
                                max_iter=5,
                                topic_word_prior=0.1,
                                doc_topic_prior=1.0,
                                learning_method='online',
                                n_jobs=-1,
                                random_state=42)

lda.fit(tfidf_matrix)

LatentDirichletAllocation(doc_topic_prior=1.0, learning_method='online',
                          max_iter=5, n_jobs=-1, random_state=42,
                          topic_word_prior=0.1)

In [42]:
# 각 댓글에 대해 토픽 분포 추출 (댓글별로 토픽 분포 계산)
topic_distributions = lda.transform(tfidf_matrix)

# 'topic_distribution' 열에 토픽 분포 저장
sample_df['topic_distribution'] = list(topic_distributions)

In [46]:
# 각 토픽의 상위 단어 출력
def print_top_words(model, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\n===== 토픽 {topic_idx + 1} =====")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

n_top_words = 10
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print_top_words(lda, tfidf_feature_names, n_top_words)


===== 토픽 1 =====
노래 별로 스타일 취향 무대 얼굴 시간 멜로디 실험 음방

===== 토픽 2 =====
데뷔 음악 노래 엔믹스 신곡 대중 진심 소녀 실망 충격

===== 토픽 3 =====
엔믹스 화이팅 노래 걸그룹 실력 조회수 권모술수 한국 라이브 활동

===== 토픽 4 =====
사랑해 설윤 지우 멤버 노래 정신 해원 엔믹스 비주얼 목소리

===== 토픽 5 =====
중독 엔믹스 쩐다 노래 얼마나 벌써 자꾸 케이팝 별별별 이상

===== 토픽 6 =====
엔믹스 릴리 컴백 파트 에스파 재능 개인 노래 핑크 퀄리티

===== 토픽 7 =====
가사 지니 대중성 안무 노래 세대 파이팅 한국인 박진영 한국어

===== 토픽 8 =====
뮤비 제왑 역대 장르 선공 본부 윤아 뮤직비디오 색깔 도대체

===== 토픽 9 =====
규진 타이틀 응원 사랑해 아이돌 분위기 기대 타이틀곡 수록곡 오해원

===== 토픽 10 =====
컨셉 노래 트와이스 보컬 듣기 매력 여름 소름 의상 최애


In [48]:
# Title별로 토픽 분포의 평균 계산
title_topic_distribution = sample_df.groupby('Title')['topic_distribution'].apply(lambda x: sum(x) / len(x))

# 정의된 토픽이름(결과 보고 이름붙임)
topic_names = {
    1: "음악성",
    2: "반응",
    3: "실력",
    4: "비주얼",
    5: "중독성",
    6: "컴백",
    7: "대중성",
    8: "예술성",
    9: "기대감",
    10: "매력"
}


# Title별 상위 세 개의 토픽 분포 출력
for title, topic_dist in title_topic_distribution.items():
    print(f"\n===== {title} 그룹의 상위 3개 토픽 분포 =====")
    
    # 토픽 분포를 (토픽 번호, 확률) 형식으로 정렬
    sorted_topic_dist = sorted(enumerate(topic_dist, 1), key=lambda x: x[1], reverse=True)
    
    # 상위 3개 토픽만 출력
    for i, prob in sorted_topic_dist[:3]:
        print(f"{topic_names[i]}: {prob:.4f}")


===== DASH 그룹의 상위 3개 토픽 분포 =====
음악성: 0.1044
컴백: 0.1025
예술성: 0.1018

===== DICE 그룹의 상위 3개 토픽 분포 =====
대중성: 0.1028
음악성: 0.1027
실력: 0.1006

===== Funky Glitter Christmas 그룹의 상위 3개 토픽 분포 =====
대중성: 0.1051
컴백: 0.1047
음악성: 0.1021

===== Love Me Like This 그룹의 상위 3개 토픽 분포 =====
음악성: 0.1069
컴백: 0.1021
실력: 0.1003

===== O.O 그룹의 상위 3개 토픽 분포 =====
음악성: 0.1034
반응: 0.1027
대중성: 0.1009

===== Party O’Clock 그룹의 상위 3개 토픽 분포 =====
매력: 0.1048
비주얼: 0.1024
음악성: 0.1018

===== Roller Coaster 그룹의 상위 3개 토픽 분포 =====
음악성: 0.1045
매력: 0.1029
비주얼: 0.1026

===== Soñar (Breaker) 그룹의 상위 3개 토픽 분포 =====
컴백: 0.1029
음악성: 0.1027
실력: 0.1020

===== Young, Dumb, Stupid 그룹의 상위 3개 토픽 분포 =====
음악성: 0.1057
비주얼: 0.1056
컴백: 0.1011

===== 별별별 (See that) 그룹의 상위 3개 토픽 분포 =====
음악성: 0.1020
예술성: 0.1014
컴백: 0.1013

===== 蜚蜚 (FEIFEI) 그룹의 상위 3개 토픽 분포 =====
음악성: 0.1156
비주얼: 0.1066
컴백: 0.1006


In [56]:
# 첫 번째 샘플의 topic_distribution에서 최대값의 인덱스를 찾기
def get_topic_name(value):
    # 토픽 분포 중 최대값의 인덱스를 찾고, 그에 해당하는 토픽 이름 반환
    idx = np.argmax(value) + 1  # 1을 더해서 topic_names에 맞는 인덱스로 변환
    return topic_names[idx]

# 댓글의 topic_distribution에서 상위 3개의 토픽 이름을 얻기
def get_topic_names(value):
    # 토픽 분포를 내림차순으로 정렬한 후 상위 3개의 인덱스를 찾고, 그에 해당하는 토픽 이름을 반환
    top_indices = np.argsort(value)[-3:][::-1]  # 상위 3개 인덱스 (내림차순으로 정렬)
    top_topics = [topic_names[idx + 1] for idx in top_indices]  # 1을 더해서 topic_names에 맞는 인덱스로 변환
    return top_topics

sample_df['topic_distribution'] = sample_df['topic_distribution'].apply(get_topic_name)

In [61]:
sample_df[['comment','topic_distribution']].head(30)

,comment,topic_distribution
0,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,중독성
1,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,매력
2,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,실력
3,이야 노래 종나 멋있네,음악성
4,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,대중성
5,노래는 좋은데 ㅋㅋ 가사는 개짜치는 별별별ㅋㅋㅋㅋ,중독성
6,해원이 외모체크 이후로 찐팬됨 멤버 모두 너무 착하고 예쁨 실력도 탑이네,비주얼
7,와 진짜 설윤이랑 지우랑 비주얼 폭발하는 노래 하나 타이틀로 내자 정말 엔믹 요즘 ...,비주얼
8,[timecode] 이어폰 까고 들으니까 무슨 연극같음,중독성
9,ㅈ구림,실력


In [ ]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)